In [1]:
# Import required libraries
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
%matplotlib inline

In [2]:
# Import required libraries for video processing
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [3]:
# Pipeline
# below routine converts image to greyscale
def convert_to_greyscale(image):
    return cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

# Gaussian filtering
def apply_Gauss(image):
    Kernel_size = 3
    return cv2.GaussianBlur(image,(Kernel_size,Kernel_size),0)

#Apply CannY Edge detection
def apply_canny(image):
    low_threshold = 40
    high_threshold = 150
    return cv2.Canny(image, low_threshold, high_threshold)

# Set region of Interest
def set_ROI(image):
    mask = np.zeros_like(image) 
    imshape = image.shape
    x1 = 150
    y1 = imshape[0]
    x2 = int(imshape[1]*0.5)
    y2 = int(imshape[0]*0.5)
    x3 = int(imshape[1]*0.5)+10
    y3 = int(imshape[0]*0.5)
    x4 = imshape[1]-10
    y4 = imshape[0]
    
    if len(image.shape) > 2:
        channel_count = image.shape[2]  # i.e. 3 or 4 depending on your image
         #   print ("channel count is ", channel_count)
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
         
    imshape = image.shape
    vertices = np.array([[(x1,y1),(x2,y2), (x3,y3),(x4,y4)]], dtype=np.int32)

    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_edges = cv2.bitwise_and(image, mask)
    return masked_edges;

#Apply Hough Transformation
def apply_hough_transformation(masked_edges,image,Y1,Y2,custom = False,smoothen=False):
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 40     # minimum number of votes (intersections in Hough grid cell) ' original 40
    min_line_length = 100 #minimum number of pixels making up a line 'original 80
    max_line_gap = 160    # maximum gap in pixels between connectable line segments 'original 10
    line_image = np.zeros((*masked_edges.shape, 3), dtype=np.uint8)
    
    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]),min_line_length, max_line_gap)
    draw_lines(line_image,lines,Y1,Y2,custom,smoothen)
    return line_image

# create a routine to process image from frame
def process_image_master(image,custom = False,smoothen = False):
    imshape = image.shape
    Y1 = int(imshape[1]*0.7)
    Y2 = int(imshape[0]*0.6)
    gray = convert_to_greyscale(image)
    blur_gray = apply_Gauss(gray)
    edges = apply_canny(blur_gray)
    masked_edges = set_ROI(edges)
    hough_output = apply_hough_transformation(masked_edges,image,Y1,Y2,custom,smoothen)
    new_masked_image = cv2.cvtColor(masked_edges,cv2.COLOR_GRAY2BGR)
    line_edges = cv2.addWeighted(image, 0.8, hough_output, 1, 0) 
      
    return line_edges 

# Draw Lines
def draw_lines(line_image,lines, Y1, Y2, custom= False,smoothen= False):
    #color=[255, 0, 0] 
    #thickness=2
    #if lines is not None:
    #    for line in lines:
    #        for x1,y1,x2,y2 in line:
    #            cv2.line(line_image, (x1, y1), (x2, y2), color, thickness)
    #    return
   # print("y1 and y2 are ", Y1,Y2)
   # Left line
    left_line = (0,0,0,0)
    left_slope, left_line_max = 0.1,0
    
    # Right line
    right_line = (0,0,0,0)
    right_slope, right_line_max = -0.1, 0
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope = ((y1-y2)/(x1-x2))
            # length of the vector from the origin to point (x, y)
            line_length = math.sqrt((x2-x1)**2 + (y2-y1)**2) # Euclidean method: sqrt(x*x + y*y)
            
        #Left lane should have +ve slope       
        if(slope > 0):
            if(line_length > left_line_max):
                left_line = (x1,y1,x2,y2)
                left_slope = slope
                left_line_max = line_length
                   
        #Right lane should have -ve slope
        elif(slope < 0):
            if(line_length > right_line_max):
                right_line = (x1,y1,x2,y2)
                right_slope = slope
                right_line_max = line_length
    
    # Computing Intercept for both lanes using Slope-intercept form:  c = -mx + y
    left_intercept = -(left_slope * left_line[0]) + left_line[1] 
    right_intercept = -(right_slope * right_line[0]) + right_line[1]    

    #Variables for a complete extrapolated line
    #Y1 = 540
    #Y2 = 330
    
    #Equation for extrapolated line
    #x = (y - c) / m
    LX1 = int((Y1 - left_intercept)/left_slope)
    LX2 = int((Y2 - left_intercept)/left_slope)
    RX1 = int((Y1 - right_intercept)/right_slope)
    RX2 = int((Y2 - right_intercept)/right_slope)

    cv2.line(line_image, (LX1,Y1), (LX2,Y2), [255,0,0], 8)
    cv2.line(line_image, (RX1,Y1), (RX2,Y2), [255,0,0], 8)
 
     



In [4]:
import os
output_path = 'test_images/output_customized/'
if not os.path.isdir(output_path):
    os.mkdir(output_path)


for x in os.listdir("test_images/"):
    input_path = "test_images/" + x
    if os.path.isdir(input_path):
        continue
    image = mpimg.imread(input_path)
    result = process_image_master(image, custom = True)

In [5]:
    # process video by converting them into frames and process using process_image_master
    def process_video(vd_video,testmode):
        final_output = "output-" + vd_video
        unprocessed_clip = VideoFileClip(vd_video)
        unprocessed_clip.save_frame("firstimage" + vd_video + ".jpeg")
        firstimage = mpimg.imread("firstimage" + vd_video + ".jpeg")
        imshape = firstimage.shape
        print("video size is - ", imshape[1],imshape[0])
        color=[255, 0, 0] 
        thickness=2
        x1 = 150
        y1 = imshape[0]
        x2 = int(imshape[1]*0.5)
        y2 = int(imshape[0]*0.5)
        x3 = int(imshape[1]*0.5)+10
        y3 = int(imshape[0]*0.5)
        x4 = imshape[1]-10
        y4 = imshape[0]
        cv2.line(firstimage,(x1,y1),(x2,y2),color,thickness)
        cv2.line(firstimage,(x2,y2),(x3,y3),color,thickness)
        cv2.line(firstimage,(x3,y3),(x4,y4),color,thickness)
        cv2.line(firstimage,(x1,y1),(x4,y4),color,thickness)
        vertices = np.array([[(x1,y1),(x2,y2), (x3,y3),(x4,y4)]], dtype=np.int32)
        if testmode == True:
            plt.imshow(firstimage)
            plt.show()
        else:
            print("verticies are ", vertices)
            processed_clip = unprocessed_clip.fl_image(process_image_master)
            %time processed_clip.write_videofile(final_output,audio=False)
    

In [6]:
process_video("challenge.mp4", False)
process_video("solidWhiteRight.mp4", False)
process_video("solidYellowLeft.mp4", False)

video size is -  1280 720
verticies are  [[[ 150  720]
  [ 640  360]
  [ 650  360]
  [1270  720]]]
[MoviePy] >>>> Building video output-challenge.mp4
[MoviePy] Writing video output-challenge.mp4


100%|██████████| 251/251 [03:38<00:00,  1.04it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output-challenge.mp4 

CPU times: user 29.7 s, sys: 1.22 s, total: 30.9 s
Wall time: 4min 2s
video size is -  960 540
verticies are  [[[150 540]
  [480 270]
  [490 270]
  [950 540]]]
[MoviePy] >>>> Building video output-solidWhiteRight.mp4
[MoviePy] Writing video output-solidWhiteRight.mp4


100%|█████████▉| 221/222 [01:38<00:00,  2.48it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output-solidWhiteRight.mp4 

CPU times: user 14.7 s, sys: 670 ms, total: 15.4 s
Wall time: 1min 48s
video size is -  960 540
verticies are  [[[150 540]
  [480 270]
  [490 270]
  [950 540]]]
[MoviePy] >>>> Building video output-solidYellowLeft.mp4
[MoviePy] Writing video output-solidYellowLeft.mp4


100%|█████████▉| 681/682 [04:18<00:00,  3.89it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output-solidYellowLeft.mp4 

CPU times: user 35.4 s, sys: 2.3 s, total: 37.7 s
Wall time: 4min 26s
